**Assignment**

Design and implement a proof-of-concept solution for building a chatbot that can
answer simple questions for customers of Electricity and Gas companies
truthfully. Please ensure that the chatbot is friendly and does not answer
questions which are not relevant.
You can make suitable assumptions to implement the PoC. For example,you may
use a PDF of user manual, or FAQ page of the company like
https://help-qa.pge.com/s/topic/0TO8M000000GxHCWA0/bill-faqs?language=en_U
S.
Make sure that you use as much open source GenAI technologies as suitable in
your solution.

# Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Google GenAI Environment & Langchain

In [2]:
!pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv
!pip install langchain
!pip install pypdf
!pip install chromadb
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Install Necessary Resources

In [3]:
from google.colab import userdata
from IPython.display import Markdown, display
import google.generativeai as genai
import os
import pandas as pd
import numpy as np

In [4]:
# Call the API Key
API_KEY = userdata.get('API_KEY')

genai.configure(api_key = API_KEY)

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=API_KEY)

# Load Document

In [6]:
from langchain.document_loaders import PyPDFLoader

pdf_file = '/content/drive/MyDrive/Colab Notebooks/Kecilin Test/billing-faqs.pdf'
Loader = PyPDFLoader(pdf_file)
pages = Loader.load_and_split()
pages

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Kecilin Test/billing-faqs.pdf', 'page': 0}, page_content='Billing\xa0FAQs \nCan\xa0I\xa0access\xa0my\xa0charges\xa0and\xa0credits\xa0online?\xa0\nWe\xa0strongly\xa0recommend\xa0enrolling\xa0in\xa0My\xa0Energy\xa0at\xa0\nwww.pge.com/myenergy\xa0for\xa0easy\xa0access\xa0to\xa0your\xa0PG&E\xa0Bill\xa0and\xa0NEM\xa0Statements.\xa0You\xa0can\xa0review\xa0your\xa0net\xa0usage\xa0online\xa0and\xa0make\xa0payments\xa0on\xa0your\xa0regular\xa0PG&E\xa0bill.\xa0Here,\xa0you’ll\xa0also\xa0find\xa0the\xa0information\xa0you\xa0need\xa0to\xa0understand\xa0and\xa0manage\xa0your\xa0energy\xa0use.\xa0\nWhy\xa0is\xa0it\xa0important\xa0to\xa0know\xa0whether\xa0I\xa0generated\xa0more\xa0energy\xa0than\xa0I\xa0used,\xa0or\xa0used\xa0more\xa0than\xa0I\xa0generated?\xa0\nWhile\xa0solar\xa0and\xa0other\xa0renewables\xa0always\xa0save\xa0you\xa0money\xa0by\xa0cutting\xa0down\xa0on\xa0your\xa0need\xa0to\xa0purchase\xa0electricity,\xa0you\xa0ca

# Clean PDF

In [7]:
cleaned_pages = []
for page in pages:
  content = page.page_content.replace('\xa0', ' ')
  cleaned_pages.append(content)

In [8]:
base_knowledge = '\n'.join(cleaned_pages)
base_knowledge = base_knowledge.strip()

print(base_knowledge)

Billing FAQs 
Can I access my charges and credits online? 
We strongly recommend enrolling in My Energy at 
www.pge.com/myenergy for easy access to your PG&E Bill and NEM Statements. You can review your net usage online and make payments on your regular PG&E bill. Here, you’ll also find the information you need to understand and manage your energy use. 
Why is it important to know whether I generated more energy than I used, or used more than I generated? 
While solar and other renewables always save you money by cutting down on your need to purchase electricity, you can go even further to maximize your benefits by understanding when and how you’re using electricity to help you make informed decisions and manage your usage. 
Why is there a small monthly charge from PG&E? 
The “Electric Monthly Charge” or “Minimum Amount Due” charge reflects a nominal amount that all customers on your selected rate schedule must pay regardless of whether or not they have solar or if any electricity is u

# Split Docs to Chunks

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50000, chunk_overlap= 0)
texts = text_splitter.split_text(base_knowledge)

In [10]:
texts

['Billing FAQs \nCan I access my charges and credits online? \nWe strongly recommend enrolling in My Energy at \nwww.pge.com/myenergy for easy access to your PG&E Bill and NEM Statements. You can review your net usage online and make payments on your regular PG&E bill. Here, you’ll also find the information you need to understand and manage your energy use. \nWhy is it important to know whether I generated more energy than I used, or used more than I generated? \nWhile solar and other renewables always save you money by cutting down on your need to purchase electricity, you can go even further to maximize your benefits by understanding when and how you’re using electricity to help you make informed decisions and manage your usage. \nWhy is there a small monthly charge from PG&E? \nThe “Electric Monthly Charge” or “Minimum Amount Due” charge reflects a nominal amount that all customers on your selected rate schedule must pay regardless of whether or not they have solar or if any electri

# Create Embeddings & Store to Vector Database

In [11]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain.docstore.document import Document

from langchain.vectorstores import Chroma


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=API_KEY)

# Convert the texts into vectors
documents = [Document(page_content=text) for text in texts]
vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings)

In [12]:
vectorstore

# Create Prompt and Role

In [26]:
from langchain import PromptTemplate

prompt_template = """
  You are a friendly chatbot for an Electric and Gas Company. Always start your response with "Thank you for asking!"
  Answer the question as detailed as possible from the provided context for the sake of customer satisfaction, make sure to provide all the details.
  If the answer is not in provided context you have to reply with, "I'm sorry, your question is not in my capabilities. Please contact the Customer Service for further assistance".
  Don't provide the wrong answer\n\n
  Context:\n {context}?\n
  Question: \n{question}\n

  Answer:
"""

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

# Create Retrieval Function

In [36]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain

retriever = vectorstore.as_retriever(k=10)

# User Input

In [38]:
question = "What is Minimum Amount Due?"

chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
docs = retriever.get_relevant_documents(question)
docs

[Document(metadata={}, page_content='Billing FAQs \nCan I access my charges and credits online? \nWe strongly recommend enrolling in My Energy at \nwww.pge.com/myenergy for easy access to your PG&E Bill and NEM Statements. You can review your net usage online and make payments on your regular PG&E bill. Here, you’ll also find the information you need to understand and manage your energy use. \nWhy is it important to know whether I generated more energy than I used, or used more than I generated? \nWhile solar and other renewables always save you money by cutting down on your need to purchase electricity, you can go even further to maximize your benefits by understanding when and how you’re using electricity to help you make informed decisions and manage your usage. \nWhy is there a small monthly charge from PG&E? \nThe “Electric Monthly Charge” or “Minimum Amount Due” charge reflects a nominal amount that all customers on your selected rate schedule must pay regardless of whether or no

# System Response

In [39]:
response = chain.invoke(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)

In [40]:
response

{'output_text': 'Thank you for asking! The “Electric Monthly Charge” or “Minimum Amount Due” charge reflects a nominal amount that all customers on your selected rate schedule must pay regardless of whether or not they have solar or if any electricity is used.'}